In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [9]:
df.head(5)

,artist,song,text
0,Hillsong,More Than,Your eternal word formed the universe \r\nAnd...
1,Gino Vannelli,Ugly Man,Who am I \r\nTo the rabble and the wise \r\n...
2,Billie Holiday,"One Never Knows, Does One?","One never knows, does one? \r\nWhen love will..."
3,George Jones,Are The Good Times Really Over (I Wish A Buck ...,I wish a buck was still silver \r\nIt was bac...
4,Louis Armstrong,Got A Bran' New Suit,Got a bran' new suit \r\nGot a bran' new tie ...


In [10]:
df['text'][0]

"Your eternal word formed the universe  \r\nAnd You speak to me whisper I'm your greatest love  \r\nYou're the saving Lord, beauty in a storm  \r\nEverything I need more than I could ever want  \r\nYou are my heart's desire  \r\nAnd I have fallen to the point of no return  \r\nMore than my heart could know  \r\nMore than in love with you  \r\nYou're the saving Lord, beauty in a storm  \r\nEverything I need more than I could ever want  \r\nYou are my heart's desire  \r\nAnd I have fallen to the point of no return  \r\nMore than my heart could know  \r\nMore than in love with you  \r\nYou are my heart's desire  \r\nAnd I have fallen to the point of no return  \r\nMore than my heart could know  \r\nMore than in love with you, with you  \r\nYou are my heart's desire  \r\nAnd I have fallen to the point of no return  \r\nMore than my heart could know  \r\nMore than in love with you  \r\n\r\n"

In [11]:
df.shape

(5000, 3)

In [13]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [14]:
import nltk

In [15]:
from nltk.stem.porter import PorterStemmer

In [16]:
stemmer=PorterStemmer()

In [17]:
def tokenization(txt):
    tokens=nltk.word_tokenize(txt)
    stemming=[stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [20]:
df['text']=df['text'].apply(lambda x: tokenization(x))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector=TfidfVectorizer(analyzer='word',stop_words='english')
matrix=tfidvector.fit_transform(df['text'])
similarity=cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.03915416, 0.08584092, ..., 0.01758421, 0.00609224,
       0.        ])

In [24]:
df[df['song']=='Ugly Man']

,artist,song,text
1,Gino Vannelli,Ugly Man,who am i to the rabbl and the wise am i just a...


In [25]:
def recommendation(song_df):
    idx=df[df['song']==song_df].index[0]
    distances=sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    songs=[]
    for m_id in distances[1:11]:
        songs.append(df.iloc[m_id[0]].song)
    return songs

In [26]:
recommendation('Ugly Man')

['The Love Of My Man',
 'Hate',
 'The Man With The Child In His Eyes',
 "Young Man's Town",
 'One True Man',
 "I'm Waiting For The Man",
 'A Year Or A Day',
 'Yes, I Will',
 'New World Man',
 "I Can't Understand It"]

In [30]:
import pickle

In [33]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [34]:
pickle.dump(df,open('df.pkl','wb'))